# Laboratorio 5: Ejemplo de ejecución paralela en CUDA

## Programa: Multiplicación escalar

El siguiente programa realiza una multiplicación escalar entre el vector `vector` y el valor escalar `scalar`. Además, se incluye una pequeña sección de comprobación de errores para verificar que la respuesta generada es correcta a través de la comparación entre los valores almacenados en los vectores `vector` y `original`.

### Implementación

In [69]:
%%writefile scavec_mult.cpp

#include <iostream>
#include <math.h>

void scalar_multiplication(int size, float *vector, float scalar)
{
  for (int i = 0; i < size; ++i)
    vector[i] *= scalar;
}

int main(void)
{
  srand(1);
  int N = 1<<20; // 1M elements

  float* vector = new float[N];
  float* original = new float[N]; // copia para validar
  float scalar = rand();

  for (int i = 0; i < N; i++){
    vector[i] = rand();
    original[i] = vector[i];
  }

  scalar_multiplication(N, vector, scalar);

  // --- Comprobación de errores ---
  bool ok = true;
  for (int i = 0; i < N; i++){
    float expected = original[i] * scalar;
    if (fabs(vector[i] - expected) > 1e-5f) {
      std::cout << "Error en índice " << i
                << ": obtenido = " << vector[i]
                << ", esperado = " << expected << "\n";
      ok = false;
      break;
    }
  }

  if (ok)
    std::cout << "Comprobación exitosa: todos los valores coinciden.\n";

  // Free memory
  delete [] vector;
  delete [] original;

  return 0;
}


Overwriting scavec_mult.cpp


### Compilación

In [70]:
%%shell
g++ scavec_mult.cpp -o scavec_mult

### Resultados

In [71]:
%%shell
./scavec_mult

Comprobación exitosa: todos los valores coinciden.


## Paralelización

Para paralelizar el programa a través de CUDA, se realizaron los siguientes cambios en base a la información consultada en Harris, M. (2025):

- Se asigna memoria de modo que el GPU pueda acceder a ella. Esto se hace a través de un modelo de programación llamado *Unified Memory*, que provee un espacio de memoria único accesible a todos los GPUs y CPUs en el sistema. Para acceder a la *Unified Memory*, se cambiaron las declaraciones de los vectores `vector` y `original` por la función CUDA `cudaMallocManaged`, y la liberación de memoria cambió al uso de la función `cudaFree`.
- La función principal `scalar_multiplication` se declara con la especificación `__global__` para convertirlo en un *kernel*, una función ejecutable por un GPU en CUDA (NVIDIA, 2025).
- Para terminar la transformación de la función `scalar_multiplication` a una función *kernel*, se declaran los enteros `index` y `stride`, que representan, respectivamente, el índice del thread en ejecución y el desplazamiento hasta el comienzo de su bloque.
- Finalmente, se hace el llamado a la nueva función *kernel* `scalar_multiplication` a través de las tres llaves angulares `<<<>>>`, en donde se incluyen las variables `blockSize` (el tamaño del bloque asignado) y `numBlocks` (el número de bloques).

In [76]:
%%writefile scavec_mult.cu

#include <iostream>
#include <math.h>

__global__
void scalar_multiplication(int size, float *vector, float scalar)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  for (int i = index; i < size; i += stride)
    vector[i] *= scalar;
}

int main(void)
{
  srand(1);
  int N = 1<<20; // 1M elements

  float* vector;
  float* original; // copia para validar

  cudaMallocManaged(&vector, N*sizeof(float));
  cudaMallocManaged(&original, N*sizeof(float));

  float scalar = rand();

  for (int i = 0; i < N; i++){
    vector[i] = rand();
    original[i] = vector[i];
  }

  int blockSize = 256;
  int numBlocks = (N + blockSize - 1) / blockSize;

  scalar_multiplication<<<numBlocks, blockSize>>>(N, vector, scalar);

  cudaDeviceSynchronize();

  // --- Comprobación de errores ---
  // Comprobar errores del kernel
  cudaError_t err = cudaGetLastError();
  if (err != cudaSuccess) {
    std::cerr << "CUDA kernel error: " << cudaGetErrorString(err) << "\n";
    return 1;
  }

  // Comprobar errores de la multiplicación
  bool ok = true;
  for (int i = 0; i < N; i++){
    float expected = original[i] * scalar;
    if (fabs(vector[i] - expected) > 1e-5f) {
      std::cout << "Error en índice " << i
                << ": obtenido = " << vector[i]
                << ", esperado = " << expected << "\n";
      ok = false;
      break;
    }
  }

  if (ok)
    std::cout << "Comprobación exitosa: todos los valores coinciden.\n";

  cudaFree(vector);
  cudaFree(original);

  return 0;
}


Overwriting scavec_mult.cu


### Compilación y resultados

In [81]:
%%shell

nvcc -arch=compute_75 -code=sm_75 scavec_mult.cu -o scavec_mult_cu
./scavec_mult_cu

Comprobación exitosa: todos los valores coinciden.


## Análisis de tiempo y entorno de ejecución

Información sobre el GPU asignado para la ejecución por Google Colab:

In [82]:
%%shell

nvidia-smi

Fri Dec  5 07:35:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Tiempo que toma el kernel para terminar la ejecución del programa:

In [83]:
%%shell

nvprof ./scavec_mult_cu

==21755== NVPROF is profiling process 21755, command: ./scavec_mult_cu
Comprobación exitosa: todos los valores coinciden.
==21755== Profiling application: ./scavec_mult_cu
==21755== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  1.7730ms         1  1.7730ms  1.7730ms  1.7730ms  scalar_multiplication(int, float*, float)
      API calls:   98.76%  195.72ms         2  97.862ms  63.057us  195.66ms  cudaMallocManaged
                    0.90%  1.7933ms         1  1.7933ms  1.7933ms  1.7933ms  cudaDeviceSynchronize
                    0.20%  389.76us         2  194.88us  117.16us  272.60us  cudaFree
                    0.07%  131.72us         1  131.72us  131.72us  131.72us  cudaLaunchKernel
                    0.06%  124.83us       114  1.0940us     104ns  50.306us  cuDeviceGetAttribute
                    0.01%  11.598us         1  11.598us  11.598us  11.598us  cuDeviceGetName
                    0.00%  5.8270u

## Referencias

- Harris, M. (2025). An Even Easier Introduction to CUDA (Updated). *NVIDIA Technical Blog*. https://developer.nvidia.com/blog/even-easier-introduction-cuda/
- NVIDIA. (2025). *2.1. Intro to CUDA C++ — CUDA Programming Guide. CUDA Programming Guide*. https://docs.nvidia.com/cuda/cuda-programming-guide/02-basics/intro-to-cuda-cpp.html